# **LENDING CLUB CREDITS**

In this project, we will study a financial issue : modeling a borrower's credit risk. For this, we use financial lending data from Lending Club, which is a marketplace for personal loans. It basically matches borrowers who are seeking a loan with investors looking to lend money and make a return.

## **How Lending Club works**

Each borrower must fill a comprehensive application containing various information, such as past financial history or the reason for the loan. Based on those data, the borrower is assigned a credit score and an interest rate according to Lending Club's own data science process. The interest rate corresponds to the percent in addition to the requested loan amount the borrower has to pay back. The higher the interest rate is, the riskier the borrower, who is unlekely to pay back the loan. A lower interest rate means higher chance that the borrower will pay the loan on time. The interest rates goes from 5.32% to 30.99% and each borrower is given a grade according to the assigned interest rate. The loan is listed on the Lending Club Marketplace once the borrower accepts the interest rate.

Investors can browse information about borrowers like the credit score and decide who they want to loan their money. Once a loan has been accepted, the borrower makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club then redistributes these payments to the investors that participated in a given loan, which means they don't have to wait until the amount is completely paid before getting a return in money.
It is important for the investors to have the loan paid back on time, so that they can make a return which corresponds to the interest rate the borrower had to pay in addition to the requested amount. Unfortunately, it is frequent to see loans that are not completely paid on time.

## **The data**

Data for all of the approved and declined loan applications are realeased periodically on the Lending Club website. It is possible to download datasets for a given year range in CSV format. Datasets come with a data dictionary in XLS format, which contains information on the different column names. It is available here : https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit


The LoanStats sheet describes the approved loans datasets and the RejectStats describes the rejected loans datasets. We focus on data about approved loans only since rejected applications don't appear on the Lending Club marketplace and aren't available for investment. The approved loans datasets contain information on current loans, completed loans, and defaulted loans.

## **Problematic**

The aim of this project is to determine which loans are more likely to be paid off from the perspective of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time.

How can we accurately predict if a borrower will pay off their loan on time ?

We will use machine learning models to answer this question. We will use approved loans data from 2007 to 2011 for a better accuracy, since most of the loans have already finished.

## **Data preparation**

We need to define a target column to make our predicitons on, and figure out which features we are going to use in our model. Let's have a look at the dataset.


In [1]:
import pandas as pd
loans_2007 = pd.read_csv("loans_2007.csv")
loans_2007.drop_duplicates()
print(loans_2007.iloc[0])

C:\Users\Galak\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

In [2]:
print(loans_2007.shape[1])

52


### **Target column definition**

The loan_status column is the only column that directly provides information on the payment of a loan. It tells if the loan was paid off on time, had delayed payments, or was defaulted on the borrower. To be able to use this column for the training model, the text needs to be converted into a numerical value.

In [3]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


To predict if a loan will get paid on time or not, we are interested if the loan has been paid or charged off. The other categories are not relevant to make the predictions because they relate to ongoing loans and can be removed.

We can use a binary classification for our variable, 1 for fully paid loans, 0 for charged off loans. However, there is a class imbalance between the positive and negative classes and this must be considered when buidling the model, as it could lead to a strong bias towards positive outcomes.

Let's remove all the values that do not matter and assign 1 and 0 to respectively paid and charged off loans.

In [4]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

### **Features selection**

Features must follow some guidelines :

  -they must not leak information from the future (after the loan has already been funded). This could cause overfitting.
  -they don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)

We should avoid features whose data are formatted poorly and need to be cleaned up, require more data or a lot of processing to turn into a useful feature, or contain redundant information.

Let's first remove the columns we don't need at first sight.

In [5]:
loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d", "zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                             1
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               J

Let's look at the retained features and check if they bring or not useful input for the model. Some features have in fact only one value and must be droped. For each column, we first eliminate NA values and then eliminate the columns containing only one value and print the result.

In [6]:
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


Let's now look for missing values.

In [7]:
null_counts = loans_2007.isnull().sum()
print(null_counts[null_counts>0])

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


The variables emp_length and pub_rec_bankruptices contain many missing values. Employment length is frequently used as an indicator to assess how risky a potential borrower is, and it should not be wise to eliminate this column. Let's check the pub_rec_bankruptcies variable.

In [8]:
print(loans_2007.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


We can see that nearly 94% of this column have the same value, which indicates a low variability, we can eliminate this column of the features set. The missing values of the other columns are scarce and we can safely remove the rows containing them only once the pub_rec_bankruptcies variable has been removed (to avoid eliminating the rows with missing values in this column). Let's also analyse the types of the remaining features.

In [9]:
loans_2007 = loans_2007.drop("pub_rec_bankruptcies", axis=1)
loans_2007 = loans_2007.dropna(axis=0)
print(loans_2007.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


The features of the "object" type must be converted to numerical data types in order to be used with the scikit-learn library. Let's create a dataframe containing only those columns.

In [10]:
object_columns_df = loans_2007.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


Some columns represent numeric values and to be converted:

int_rate: interest rate of the loan in %
revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit

Some features seem to be categorical variables, we use the dictionnary to confirm and check how many categories they have :

home_ownership: home ownership status, can only be 1 of 4 categorical variables
verification_status: indicates if income was verified by Lending Club
emp_length: number of years the borrower was employed upon time of application
term: number of payments on the loan, either 36 or 60
addr_state: borrower's state of residence
purpose: a category provided by the borrower for the loan request
title: loan title provided by the borrower

The columns purpose and title seem to reflect the same information. Let's explore the unique value counts separately to confirm if this is true, and drop one of those columns accordingly.

Lastly, some of the columns contain date values that require considerable engineering before being potentially useful:

earliest_cr_line: The month the borrower's earliest reported credit line was opened
last_credit_pull_d: The most recent month Lending Club pulled credit for this loan

In [11]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans_2007[c].value_counts())
print("\n")
print(loans_2007["title"].value_counts())
print("\n")
print(loans_2007["purpose"].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
LA     420
AL     420
KY     311
OK     285
UT     249
KS     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
AK      76
WY      76
SD      60
VT  

The emp_length column should be cleaned and treated as a numerical feature since the values have ordering. THis implies making some assumptions : people having worked less than one year are considered as not having worked, and people having worked more than 10 years are considered to have only really worked 10 years. It is fine to make those assumptions in our case, since we consider the perspective of conservative investors that prefers missing out opportunities than risking unpaid loans. We use a dictionary in order to use the replace function to create the new numerical categories.

The home_ownership, verification_status, and term columns each contain a few discrete categorical values and should converted to dummy variables.

It seems like the purpose and title columns do contain overlapping information. We keep the purpose column as it contains less discrete values than the title column, which additionally shows some data quality issues (many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

We also remove the addr_state column as it contains multiple values that would make it too heavy for dummy conversion within the dataframe.

Let's directly drop the earliest_cr_line and last_credit_pull_d columns since the amount of information they bring might not be as useful in comparison to the time spent on engineering them.

In [12]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans_2007 = loans_2007.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
loans_2007["int_rate"] = loans_2007["int_rate"].str.rstrip("%").astype("float")
loans_2007["revol_util"] = loans_2007["revol_util"].str.rstrip("%").astype("float")
loans_2007 = loans_2007.replace(mapping_dict)


cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans_2007[cat_columns])
loans_2007 = pd.concat([loans_2007, dummy_df], axis=1)
loans_2007 = loans_2007.drop(cat_columns, axis=1)

## **MODEL**

### **Error metric**

The data are now prepared to feed the model. Before choosing the adequate model, we must select an error metric to evaluate how the model performs. Let's have a quick recap on our problematic : we want to enable enough loans that will be paid off on time to be able to gain the interests and have a positive balance over money loss due to unpaid loans. The error metric should help us determine if our algorithm will make the investors win or lose money. 

We are trying to predict the binary loan_status variable, which enables missclassifications issues : false positive and false negative. Since we took the perspective of conservative investors, who prefer missing out on opportunities (false negatives) than funding a risky loan (false positives), we should carefully consider the latter. Let's also remember that we are dealing with imbalance data that would tend to favor false positives.

Therefore, accuracy might not be the best metric to measure our model's performance, but rather the false and true positives rates (fpr and tpr respectively), defined as follow :

fpr = fp / (fp + tn)

tpr = tp / (tp + fn)

with: 
tp = true positives
tn = true negatives
fp = false positive
fn = false negative

Logistic regression is a good model to start when dealing with binary classification problems. To adress the imbalance between positive and negative outcomes, we set class_weight to "balanced" when creating the LogisticRegression instance in order to penalize the misclassification of the minority class during the training process.


Let's now start our prediction attempt with a logistic regression model, adapted for binary classification problems. We create a dataframe containing all the previously selected features, and a serie containing the target variable "loan_status". To avoid overfitting, we use a k-fold cross validation on the logistic regression model. We then calculate the fpr and tpr.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
# Creation of features dataframe and target serie
cols = loans_2007.columns
train_cols = cols.drop("loan_status")
features = loans_2007[train_cols]
target = loans_2007["loan_status"]

# Performing k-fold cross validation on the logistic regression model with balanced class_weight
lr = LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans_2007["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans_2007["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans_2007["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans_2007["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

C:\Users\Galak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Galak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

0.5465718405873099
0.5237907206317868


C:\Users\Galak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


We get more than 50% for true and false positives rates. While it's good for true positive, it is still quite high for false positive. Let's try to decrease the false positive rate by manually tweaking the class_weight penalty.

In [26]:
# Setting penalty values for the class-weight argument
penalty = {
    0: 10,
    1: 1
}

#Performing k-fold cross validation on the logistic regression model with manual values for the class_weight argument
lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans_2007["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans_2007["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans_2007["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans_2007["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

C:\Users\Galak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.1704902988987939
0.16722606120434352


C:\Users\Galak\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Manually changing the penalty values considerably reduced the false positive rate, but also came with the cost of reducing the true positives rate. We could keep experimenting those penalty values to reach a better compromise, but it might be useful to test another model to make our predictions.

Let's test the randomforestclassifier model on our data. We set random_state to 1 to avoid variation of the predictions due to random chance, and set the class_weight with the previously used penalty values.

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

# Setting penalty values for the class-weight argument
penalty = {
    0: 10,
    1: 1
}

# Performing k-fold cross validation on the random forest model with manual penalty values class_weight
rf = RandomForestClassifier(class_weight=penalty, random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans_2007["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans_2007["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans_2007["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans_2007["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.9964931830099633
0.9970384995064165


Unfortunately, the random forest model did not bring any improvment over the logistic regression model. It could be fixed by applying a harsher penalty for misclassifications of 0s.

**Conclusion**

Ultimately, our best model had a false positive rate of nearly 16%, and a true positive rate of nearly 17%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 16% of borrowers defaulting, and that the pool of 17% of borrowers is large enough to make enough interest money to offset the losses.

This is not optimum, and there's still room to improve. Some next steps could be :

Tweak the penalties further.
Try models other than a random forest and logistic regression.
Ensemble multiple models to get more accurate predictions.
Tune the parameters of the algorithm to achieve higher performance.